In [98]:
!pip install nltk


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [99]:
import warnings
warnings.filterwarnings("ignore")

In [100]:
import csv                               # csv reader
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support # to report on precision and recall
import numpy as np
import re

In [101]:
def StopWordsFromFile(filename):
    stop_words = []
    with open(filename) as file:
        for i in csv.reader(file):
            stop_words.append(i[0].lower())
    return stop_words

def StopWordRemoval(text, lexicon):
    text = text
    for i in lexicon:
        while i in text:
            text.remove(i)
    return text

In [102]:
def FamousPeopleFromFile(filename):
    famous_people=[]
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        for i in reader:
            famous_people.append(i[0].lower())
    return famous_people

def FamousPeopleTransform(text,lexicon):
    famous_matches = []
    text = text
    for i in lexicon:
        if i in text:
            text = re.sub(f"(^|\W){i}", f" |{len(famous_matches)} ", text)
            famous_matches.append(i)
    return [text, famous_matches]

In [103]:
def CaseFold(text):
    return text.lower()

In [104]:
def nGram(text,n=[1]):
    n = [1] + n
    text = text
    ngrams = []

    n.sort()
    for i in range(len(n)):
        start = []
        end = []
        if n[i] == 1:
            continue
        for v in range(n[i]-n[i-1]):
            start.append("<s>")
            end.append("</s>")
        # start = ["<s>" for v in range(i-1-count)]
        # end = ["</s>" for v in range(i-1-count)]
        text = start + text + end
        for j in range(len(text)-n[i]+1):
            ngrams.append(" ".join(text[j:j+n[i]]))
    return text+ngrams  

In [105]:
def NegationWordsFromFile(filename):
    negation_words = []
    with open(filename) as file:
        for i in csv.reader(file):
            negation_words.append(i[0].lower())
    return negation_words

def Negate(text, lexicon):
    text = text
    negation_words = lexicon
    for i in negation_words:
        if i in text:
            matches=[]
            for k in range(len(text)):
                if text[k] == i:
                    matches.append(k)
            for m in matches:
                for j in range(m+1, len(text)):
                    if (text[j]=="not" or "|" in text[j] ):
                        continue
                    text[j] = "not_" + text[j]
                    last = text[j][-1]
                    if (text[j][:8]=="not_not_"):
                        text[j] = text[j][8:]
                    if (text[j][-1]=="." or text[j][-1]=="!" or text[j][-1]=="?"):
                        break
        if text[-1] == "not_" or text[-1] == "":
            text.pop()
    return text

In [106]:
stop_words = StopWordsFromFile("stop_words.csv")
famous_people = FamousPeopleFromFile("famous_people.csv")
negation_words = NegationWordsFromFile("negation.csv")

In [107]:
def load_data(path):
    """Load data from a tab-separated file and append it to raw_data."""
    with open(path,encoding="utf-8") as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            if line[0] == "Id":  # skip header
                continue
            (label, data) = parse_data_line(line)
            raw_data.append((data, label))

def split_and_preprocess_data(percentage,Case_Folding=True,Punctuation=True,Stop_Word_Removal=False,Famous_People=False,nGramList=[1],Negation_Words=False,
                              Average_Word_Length=False,Types_Per_Tokens=False,Sentiment=False, Mutual_Information="all",
                             s1=False,s2=False,s3=False,s4=False,s5=False,s6=False,s7=False,s8=False,s9=False,s10=False):
    """Split the data between train_data and test_data according to the percentage
    and performs the preprocessing."""
    num_samples = len(raw_data)
    num_training_samples = int((percentage * num_samples))
    for (data, label) in raw_data[:num_training_samples]:
        train_data.append((to_feature_vector(pre_process(data[0],Case_Folding=Case_Folding, Punctuation=Punctuation, Stop_Word_Removal=Stop_Word_Removal,Famous_People=Famous_People, nGramList=nGramList,
                                                         Negation_Words=Negation_Words), data[1:],
                                             Average_Word_Length=Average_Word_Length,Types_Per_Tokens=Types_Per_Tokens,Sentiment=Sentiment,
                                             Mutual_Information=Mutual_Information,s1=s1,s2=s2,s3=s3,s4=s4,s5=s5,s6=s6,s7=s7,s8=s8,s9=s9,s10=s10),label))
    for (data,label) in raw_data[num_training_samples:]:
        test_data.append((to_feature_vector(pre_process(data[0],Case_Folding=Case_Folding, Punctuation=Punctuation, Stop_Word_Removal=Stop_Word_Removal,Famous_People=Famous_People, nGramList=nGramList,
                                                        Negation_Words=Negation_Words), data[1:],
                                             Average_Word_Length=Average_Word_Length,Types_Per_Tokens=Types_Per_Tokens,Sentiment=Sentiment,
                                             Mutual_Information=Mutual_Information,s1=s1,s2=s2,s3=s3,s4=s4,s5=s5,s6=s6,s7=s7,s8=s8,s9=s9,s10=s10),label))
        
## pre_process(text,Case_Folding=True,Punctuation=True, Stop_Word_Removal=False,Famous_People=False,nGramList=[1],Negation_Words=False):
## to_feature_vector(tokens, Average_Word_Length=False, Types_Per_Tokens=False,Sentiment=False,Mutual_Information=False):

In [108]:
def convert_label(label):
    """Converts the multiple classes into two,
    making it a binary distinction between fake news and real."""
    #return label
    # Converting the multiclass labels to binary label
    labels_map = {
        'true': 'REAL',
        'mostly-true': 'REAL',
        'half-true': 'REAL',
        'false': 'FAKE',
        'barely-true': 'FAKE',
        'pants-fire': 'FAKE'
    }
    return labels_map[label]


def parse_data_line(data_line):
    # Should return a tuple of the label as just FAKE or REAL and the statement
    # e.g. (label, statement)
    # data = [conver_label(data_line[1])]
    return (convert_label(data_line[1]),data_line[2:])

In [110]:
# Input: a string of one statement
def pre_process(text,Case_Folding=False,Punctuation=False, Stop_Word_Removal=False,Famous_People=False,nGramList=[1],Negation_Words=False):
    ## Using famous_people, stop_words, negation_words defined above
    text = text
    # Should return a list of tokens
    # DESCRIBE YOUR METHOD IN WORDS
    ## Case Folding
    if Case_Folding==True:
        text = CaseFold(text)
    ## Famous People Matching
    if Famous_People==True:
        text,famous_matches = FamousPeopleTransform(text,famous_people)
    ## RegExp Pattern Match
    if Punctuation ==True:
        pattern = "[^a-zA-Z0-9?!\'\.\|]+"
    else:
        pattern="\W"
    text = re.split(pattern, text)
    ## Stop Words
    if Stop_Word_Removal == True:
        text = StopWordRemoval(text, stop_words)
    ## Negation Words
    if Negation_Words == True:
        text = Negate(text, negation_words)
    ## Keep Punctuation
    text = " ".join(text)
    replace_pattern = "([\.\?\!])"
    for match in re.finditer(replace_pattern, text):
        text = text.replace(match.group(), f" {match.group()} ")
    text = text.split(" ")
    ## Famous Again
    if Famous_People==True and famous_matches != None:
        for i in range(len(text)):
            if len(text[i])>1 and text[i][0] =="|":
                text[i] = famous_matches[int(text[i][1])]
    ##
    while "" in text:
        text.remove("")
    while "not_" in text:
        text.remove("not_")   
    ## n-Gram
    text = nGram(text, nGramList)
    return text

In [111]:
def AverageWordLength(tokens):
    smn = 0
    for i in tokens:
        smn+= len(i)
    return smn/len(tokens)

In [112]:
def Unique_Word_Per_Total_Words(tokens):
    unique = len(set(tokens))
    total_words = len(tokens)
    return unique/total_words

In [113]:
def SentimentLexiconFromFile(filename):
    positive_sentiment = []
    negative_sentiment = []
    with open(filename) as file:
        for i in csv.reader(file):
            if i[0] == "positive":
                positive_sentiment.append(i[1].lower())
            elif i[0] == "negative":
                negative_sentiment.append(i[1].lower())
    return [negative_sentiment, positive_sentiment]

def getSentiment(tokens, negative_lexicon,positive_lexicon):
    sentiment = {"Positive Sentiment":0, "Negative Sentiment":0}
    for token in tokens:
        if token in negative_lexicon:
            sentiment["Negative Sentiment"]+=1
        elif token in positive_lexicon:
            sentiment["Positive Sentiment"]+=1
    return sentiment

In [114]:
negative_sentiment,positive_sentiment = SentimentLexiconFromFile("sentiment_lexicon.csv")

In [115]:
## Do it again but now with feature selection

def MutualInformationFeatures(filename):
    features = []
    with open(filename) as file:
        reader = csv.reader(file)
        next(reader)
        for i in reader:
            features.append(i[0])
    return features

MI_FAKE = MutualInformationFeatures("Q5_pointwise_mutual_information_FAKE.csv")
MI_REAL = MutualInformationFeatures("Q5_pointwise_mutual_information_REAL.csv")
MI_Features = list(set(MI_FAKE + MI_REAL))
print(len(MI_FAKE))
print(len(MI_REAL))
print(len(MI_Features))

6236
8878
13962


In [116]:
global_feature_dict = {} # A global dictionary of features

def to_feature_vector(tokens,rest, Average_Word_Length=False, Types_Per_Tokens=False,Sentiment=False,Mutual_Information="all",
                      s1=False,s2=False,s3=False,s4=False,s5=False,s6=False,s7=False,s8=False,s9=False,s10=False,):
    # Should return a dictionary containing features as keys, and weights as values
    # DESCRIBE YOUR METHOD IN WORDS
    features_dict = {}
    
    if s1==True:
        features_dict["s1"] = rest[0]
    if s2==True:
        features_dict["s2"] = rest[1]
    if s3==True:
        features_dict["s3"] = rest[2]
    if s4==True:
        features_dict["s4"] = rest[3]
    if s5==True:
        features_dict["s5"] = rest[4]
    if s6==True:
        features_dict["s6"] = rest[5]
    if s7==True:
        features_dict["s7"] = rest[6]
    if s8==True:
        features_dict["s8"] = rest[7]
    if s9==True:
        features_dict["s9"] = rest[8]
    if s10==True:
        features_dict["s10"] = rest[9]
    ## Average word length
    if Average_Word_Length == True:
        features_dict["Average Word Length"] = AverageWordLength(tokens)
    ## Unique_Words/Total_Words
    if Types_Per_Tokens==True:
        features_dict["Unique_Words/Total_Words"] = Unique_Word_Per_Total_Words(tokens)
    ## Sentiment Lexicon
    if Sentiment == True:
        features_dict.update(getSentiment(tokens, negative_sentiment,positive_sentiment))
    ## Mutual_Information Maker
    if Mutual_Information != "all":
        MI_data = MI_Features[:Mutual_Information]
    for token in tokens:
        if Mutual_Information != "all":
            
            if not(token in MI_data):
                continue
        features_dict.setdefault(token, 0)
        global_feature_dict.setdefault(token,0)
        features_dict[token] += 1
        global_feature_dict[token] += 1
    return features_dict

In [117]:
# TRAINING AND VALIDATING OUR CLASSIFIER


def train_classifier(data,SVC_HP={}):
    pipeline =  Pipeline([('svc', LinearSVC(**SVC_HP))])
    return SklearnClassifier(pipeline).train(data)

In [118]:
# PREDICTING LABELS GIVEN A CLASSIFIER

def predict_labels(samples, classifier):
    """Assuming preprocessed samples, return their predicted labels from the classifier model."""
    return classifier.classify_many(samples)

def predict_label_from_raw(sample, classifier):
    """Assuming raw text, return its predicted label from the classifier model."""
    return classifier.classify(to_feature_vector(preProcess(reviewSample)))

In [119]:
#solution
from sklearn.metrics import classification_report


def cross_validate(dataset, folds,return_results=False,print_results=True, pointwiseMI=False,SVC_HP={}):
    results = []
    ## 
    fp = []
    fp_dict = {}
    fn = []
    fn_dict = {}
    tp = []
    tp_dict = {}
    tn = []
    tn_dict = {}
    
    if pointwiseMI ==True:
        smoothing_factor = 0
        entropy_c = 0
        features_with_count = {}
        features_real = {}
        features_fake = {}
        information_gain = {}
        mutual_information = {}
    ## 
    ##
    cv_results = {
               "FAKE":{"precision":0, "recall":0, "f1-score":0},
               "REAL":{"precision":0, "recall":0, "f1-score":0},
               "accuracy":0,
               "macro avg":{"precision":0, "recall":0, "f1-score":0},
               "weighted avg":{"precision":0, "recall":0, "f1-score":0}
              }
    
    results_headers = list(cv_results.keys())
    results_sub_headers = list(cv_results["FAKE"].keys())
    ##
    
    fold_size = int(len(dataset)/folds) + 1 
    for i in range(0,len(dataset),int(fold_size)):
        # insert code here that trains and tests on the 10 folds of data in the dataset
        if print_results ==True:
            print("Fold start on items %d - %d" % (i, i+fold_size))
        # FILL IN THE METHOD HERE
        ##
        test = dataset[i:i+fold_size]
        test_x = []
        test_y = []
        for data in test:
            test_x.append(data[0])
            test_y.append(data[1])
        
        train = dataset[0:i]
        train.extend(dataset[i+fold_size:])
        model = train_classifier(train,SVC_HP)
        predicted = predict_labels(test_x, model)
        report = classification_report(test_y, predicted, output_dict=True)
        
        ##
        ##
        
        if i == 0:         
            if pointwiseMI == True:
                ## Counts
                real_prob = test_y.count("REAL")/len(test_y)
                fake_prob = test_y.count("FAKE")/len(test_y)

                for value in range(len(test_y)):
                    for k in list(test_x[value].keys()):
                        features_with_count.setdefault(k,smoothing_factor)
                        features_real.setdefault(k,smoothing_factor)
                        features_fake.setdefault(k,smoothing_factor)
                        features_with_count[k] += 1
                        if test_y[value] == "REAL":
                            features_real[k] += 1
                        elif test_y[value] == "FAKE":
                            features_fake[k] += 1

                feature_count_classes = sum(features_with_count.values())
                feature_count_real = sum(features_real.values())
                feature_count_fake = sum(features_fake.values())

                ##
                ## Mutual Information

                # These dictionaries contain the mutual information for features that occur in that class. This means that I don't have infinities.
                # These infinities arise when when we do MLE and the probability in Pointwise mutual information in the numerator in the log is 0.
                # This means that feature hasn't occured in that class and thus it is a good discriminator of between the classes. So these features are the best.
                # But using the mutual information results directly with infinities isn't that useful. It doesn't tell me the best features for that particular class.
                # So I will print to mr, mf, and mutual_information.
                # Most likely, the best features to keep are the ones that only occur in REAL or FAKE but not both.

                mr = {}
                mf = {}
                mr_removed = {}
                mf_removed = {}

                for feature in list(features_with_count.keys()):
                    if features_real[feature]>0:
                        mr[feature] = np.log2((features_real[feature]/feature_count_real)/((features_with_count[feature]/feature_count_classes)*(real_prob)))
                    if features_fake[feature]>0:
                        mf[feature] = np.log2((features_fake[feature]/feature_count_fake)/((features_with_count[feature]/feature_count_classes)*(fake_prob)))
                    mutual_information.setdefault(feature,0)
                    mutual_information[feature] = fake_prob*(mf[feature] if feature in mf else -np.inf) + real_prob*(mr[feature] if feature in mr else -np.inf)

                with open("Q6_pointwise_mutual_information_FAKE.csv", "w") as file:
                    header = ["Feature", "Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mf_sorted = {k: v for k, v in sorted(mf.items(), key=lambda item: -item[1])}
                    for feature,ig in mf_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})

                with open("Q6_pointwise_mutual_information_REAL.csv", "w") as file:
                    header = ["Feature", "Pointwise Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mr_sorted = {k: v for k, v in sorted(mr.items(), key=lambda item: -item[1])}
                    for feature,ig in mr_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})

                with open("Q6_average_pointwise_mutual_information.csv", "w") as file:
                    header = ["Feature", "Average Pointwise Mutual Information"]
                    writer = csv.DictWriter(file, fieldnames=header)
                    writer.writeheader()
                    mutual_information_sorted = {k: v for k, v in sorted(mutual_information.items(), key=lambda item: -item[1])}
                    for feature,ig in mutual_information_sorted.items():
                        writer.writerow({header[0]:feature, header[1]: ig})
    
        
        ##
        ## Appending the results for all
        results.append(report)
        
        for header in results_headers:
            if header == "accuracy":
                cv_results[header] += report["accuracy"]/folds
                continue
            for sub in results_sub_headers:
                cv_results[header][sub] += report[header][sub]/folds
    if print_results==True:
        for header in results_headers:
            if header == "accuracy":
                print(f"## {header} ## {cv_results[header]}")
                continue
            print(f"## {header} ##")
            for sub in results_sub_headers:
                print(f"{sub} : {cv_results[header][sub]}")
        ##
    return cv_results if return_results==True else None

In [120]:
Features = {"Case_Folding":True, "Punctuation":False,"Stop_Word_Removal":False,
            "nGramList":[5], "Negation_Words":False, "Average_Word_Length":True,
            "Types_Per_Tokens":True,"Sentiment":True,"Mutual_Information":"all"}
HP = {"loss":"hinge","C":0.2}

testing = [{"s1":True},{"s2":True},{"s3":True},{"s4":True},{"s5":True},{"s6":True},
          {"s7":True},{"s8":True},{"s9":True},{"s10":True}]

results_data =[]

raw_data = []
train_data = []
test_data = []

for i in testing:
    print(len(results_data))
    
    n_features = Features.copy()
    n_features.update(i)
    
    raw_data = []
    train_data = []
    test_data = []

    
    data_file_path = 'fake_news.tsv'

    load_data(data_file_path) 
    split_and_preprocess_data(0.8, **n_features)
    results_data.append({"parameter":i,"score":cross_validate(train_data, 10, print_results=False,return_results=True, pointwiseMI=True, SVC_HP=HP)})

0
1
2
3
4
5
6
7
8
9


In [121]:
with open("Q6_New_Feature_Analysis.csv","w") as file:
    header = ["Feature", "FAKE precision","FAKE recall","FAKE f1-score",
              "REAL precision", "REAL recall", "REAL f1-score",
              "accuracy","macro avg precision", "macro avg recall", "macro avg f1-score",
              "weighted avg precision", "weighted avg recall", "weighted avg f1-score"]
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    for i in results_data:
        t = {"Feature":i["parameter"], 
             "FAKE precision":i["score"]["FAKE"]["precision"],
             "FAKE recall":i["score"]["FAKE"]["recall"],
             "FAKE f1-score":i["score"]["FAKE"]["f1-score"],
             "REAL precision":i["score"]["REAL"]["precision"],
             "REAL recall":i["score"]["REAL"]["recall"],
             "REAL f1-score":i["score"]["REAL"]["f1-score"],
             "accuracy":i["score"]["accuracy"],
             "macro avg precision":i["score"]["macro avg"]["precision"],
             "macro avg recall":i["score"]["macro avg"]["recall"],
             "macro avg f1-score":i["score"]["macro avg"]["f1-score"],
             "weighted avg precision":i["score"]["weighted avg"]["precision"],
             "weighted avg recall":i["score"]["weighted avg"]["recall"],
             "weighted avg f1-score":i["score"]["weighted avg"]["f1-score"]
            }
        writer.writerow(t)

In [122]:
Features = {"Case_Folding":True, "Punctuation":False,"Stop_Word_Removal":False,
            "nGramList":[5], "Negation_Words":False, "Average_Word_Length":True,
            "Types_Per_Tokens":True,"Sentiment":True,"Mutual_Information":"all",
            "s1":True,"s2":True,"s3":True,"s4":True,"s5":True,"s6":True,
            "s7":True,"s8":True,"s9":True,"s10":True}
HP = {"loss":"hinge","C":0.2}


raw_data = []
train_data = []
test_data = []
    
data_file_path = 'fake_news.tsv'

load_data(data_file_path) 
split_and_preprocess_data(0.8, **Features)
cross_validate(train_data, 10, print_results=True, SVC_HP=HP)

Fold start on items 0 - 820
Fold start on items 820 - 1640
Fold start on items 1640 - 2460
Fold start on items 2460 - 3280
Fold start on items 3280 - 4100
Fold start on items 4100 - 4920
Fold start on items 4920 - 5740
Fold start on items 5740 - 6560
Fold start on items 6560 - 7380
Fold start on items 7380 - 8200
## FAKE ##
precision : 0.6756932927490217
recall : 0.6466828827641089
f1-score : 0.6605444127663895
## REAL ##
precision : 0.736498264852891
recall : 0.7610506785707933
f1-score : 0.7483504774939539
## accuracy ## 0.7112207136849693
## macro avg ##
precision : 0.7060957788009563
recall : 0.7038667806674511
f1-score : 0.7044474451301718
## weighted avg ##
precision : 0.7106892477205656
recall : 0.7112207136849693
f1-score : 0.7104308831853742


In [134]:
from sklearn.metrics import accuracy_score

# Finally, check the accuracy of your classifier by training on all the traning data
# and testing on the test set
# Will only work once all functions are complete
functions_complete = True  # set to True once you're happy with your methods for cross val
if functions_complete:
    print(test_data[0])   # have a look at the first test data instance
    classifier = train_classifier(train_data,SVC_HP=HP)  # train the classifier
    test_true = [t[1] for t in test_data]   # get the ground-truth labels from the data
    test_pred = predict_labels([x[0] for x in test_data], classifier)  # classify the test data to get predicted labels
    print("Done training!")
    print("\n\n")
    print(f"Accuracy: ",accuracy_score(test_true,test_pred))
    print("\n\nMacro:")
    final_scores = precision_recall_fscore_support(test_true, test_pred, average='macro') # evaluate
    print("Precision: %f\nRecall: %f\nF Score:%f" % final_scores[:3])
    
    
    final_scores = precision_recall_fscore_support(test_true, test_pred, average='weighted') # evaluate    

    print("\n\nWeighted:")
    print("Precision: %f\nRecall: %f\nF Score:%f" % final_scores[:3])

({'s1': 'bush-administration,deficit,taxes', 's2': 'dennis-kucinich', 's3': 'U.S. representative', 's4': 'Ohio', 's5': 'democrat', 's6': '1', 's7': '3', 's8': '4', 's9': '6', 's10': '0', 'Average Word Length': 13.210526315789474, 'Unique_Words/Total_Words': 0.8157894736842105, 'Positive Sentiment': 1, 'Negative Sentiment': 0, '<s>': 4, 'the': 2, 'bush': 1, 'tax': 1, 'cuts': 1, 'helped': 1, 'to': 1, 'create': 1, 'a': 1, 'substantial': 1, 'part': 1, 'of': 1, 'deficit': 1, '</s>': 4, '<s> <s> <s> <s> the': 1, '<s> <s> <s> the bush': 1, '<s> <s> the bush tax': 1, '<s> the bush tax cuts': 1, 'the bush tax cuts helped': 1, 'bush tax cuts helped to': 1, 'tax cuts helped to create': 1, 'cuts helped to create a': 1, 'helped to create a substantial': 1, 'to create a substantial part': 1, 'create a substantial part of': 1, 'a substantial part of the': 1, 'substantial part of the deficit': 1, 'part of the deficit </s>': 1, 'of the deficit </s> </s>': 1, 'the deficit </s> </s> </s>': 1, 'deficit </